# **資料前處理**

**安裝套件**

In [0]:
import pandas as pd
import numpy as np
import sys   #reload()之前必須要引入模組
from numpy import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
import random

**載入資料**

In [0]:
!curl -O http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip
!cd ml-100k/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4808k  100 4808k    0     0  10.0M      0 --:--:-- --:--:-- --:--:-- 10.0M
Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test       

In [0]:
data_fields = ['user id','item id', 'rating', 'timestamp']
user_fields = ['user id', 'age', 'gender', 'occupation', 'zip code']
data_df=pd.read_table("ml-100k/u.data",names=data_fields)

In [0]:
util_df=pd.pivot_table(data=data_df,values='rating',index='user id',columns='item id')

util_df

item id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**將遺失值填補為0**

In [0]:
movie=util_df.fillna(0)
movie

item id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
data=movie.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


# **Basic model**


$ \hat\gamma_{ui} = q^T _i  p_u  $

$min_{q^*,p^*}\sum_{(u,i)\in\kappa }{(\gamma_{ui}-q^T _i  p_u)^2} +\lambda (||q_i||^2+||p_u||^2)$

其中：

$\gamma$ : user-item rating  $\quad q_i$ :  item possesses those factors $\quad p_u$ :  interest the user has in items that are high on the corresponding factors

## **SGD**

$e_{ui}=\gamma_{ui} - q^T _i  p_u $
*   $q_i \leftarrow q_i+\gamma \cdot (e_{ui}\cdot p_u - \lambda \cdot q_i)$
*   $p_u \leftarrow p_u+\gamma \cdot (e_{ui}\cdot q_i - \lambda \cdot p_u)$

其中 $\gamma$ : learning rate

**寫偏微分的function**

In [0]:
def partial_residual(w,b,i,data):
  zero=np.zeros(data.shape)
  zero[row[i]][column[i]]=1
  partialresidual=zero*(data[row[i]][column[i]]-np.dot(w.T, b))
  return partialresidual

In [0]:
def partial_error(w,b,i,lamb,data):
  total=0
  row,column=data.nonzero()
  partialerror=data[row[i]][column[i]]-w[0][row[i]]*b[0][column[i]]
  return partialerror

**寫一階梯度的function**

In [0]:
def partial_step_gradient1(w,b,data,i,learning_rate,lamb):
    w_gradient=(partial_residual(w,b,i,data).dot(b.T).T-w*lamb)
    b_gradient=(partial_residual(w,b,i,data).T.dot(w.T).T-b*lamb)
    new_w=w+(w_gradient*learning_rate)
    new_b=b+(b_gradient*learning_rate)
    return [new_w,new_b]

**寫square error loss + regularization**

In [0]:
def error(w,b,lamb,data):
    totalError=0
    matric=w.T.dot(b)
    for i in range(0,data.shape[0]):
        for j in range(0,data.shape[1]):
            if data[i][j]!=0:
                totalError+=(data[i][j]-matric[i][j])**2
    totalError+=lamb*(w.dot(w.T)+b.dot(b.T))
    return totalError[0][0]

**寫 RMSE function**

In [0]:
def rmse(data, Q, P):
    I = data != 0  # Indicator function which is zero for missing data
    ME = I * (data - np.dot(Q.T, P))  # Errors between real and predicted ratings
    MSE = ME**2  
    return np.sqrt(np.sum(MSE)/np.sum(I))  # sum of squared errors

**寫SGD function**

In [0]:
def partial_gradient_descent_runner1(data,starting_w,starting_b,learning_rate,lamb,max_iterations):
    b=starting_b
    w=starting_w
    lasterror = 0
    row,column=data.nonzero()
    zero=np.zeros(data.shape)
    partial_root_mean_square_error=[]
    for j in range(max_iterations):
      for i in range(0,100000):
        zero=np.zeros(data.shape)
        w,b=partial_step_gradient1(w,b,data,i,learning_rate,lamb)
      partial_root_mean_square_error.append(rmse(data, w, b))
      print("The {}-th iteration rmse = {}".format(j+1,rmse(data, w, b)))
      print("------------------------------")
      if np.abs(error(w,b,lamb,data)-lasterror)>0.1:
        lasterror=error(w,b,lamb,data)
      else:
        break
    return partial_root_mean_square_error

**模擬 Matrix Factorization by SGD**

In [0]:
w=np.array([[0]*data.shape[0]])
b=np.array([[0]*data.shape[1]])
lamb=0.01
learning_rate=0.0004
max_iterations=1
row,column=data.nonzero()
## partial_root_mean_square=partial_gradient_descent_runner1(data,w,b,learning_rate,lamb,max_iterations)

## **Min-batch**

In [0]:
def minbatch_partial_residual(w,b,i,minbatch,data):
  random.seed(10)
  group_index=[[row[l],column[l]] for l in range(len(row))]
  v=0
  group=[]
  while v < row.shape[0]:
    group.append(group_index[v:v+minbatch])
    v+=minbatch
  group=np.array(group)
  ##group[k][l][m]=> k-th group l-th data m=0 =>row m=1 =>column

  zero=np.zeros(data.shape)
  data_zero=np.zeros(data.shape)
  for k in range(group[i].shape[0]):
    zero[group[i][k][0]][group[i][k][1]]=1
    data_zero[group[i][k][0]][group[i][k][1]]=data[group[i][k][0]][group[i][k][1]]
  minbatch_partialresidual=zero*(data_zero-np.dot(w.T, b))
  return minbatch_partialresidual

In [0]:
def minbatch_partial_step_gradient1(w,b,data,i,learning_rate,lamb):
    w_gradient=(minbatch_partial_residual(w,b,i,minbatch,data).dot(b.T).T-w*lamb)
    b_gradient=(minbatch_partial_residual(w,b,i,minbatch,data).T.dot(w.T).T-b*lamb)
    new_w=w+(w_gradient*learning_rate)
    new_b=b+(b_gradient*learning_rate)
    return [new_w,new_b]

In [0]:
def error(w,b,lamb,data):
    totalError=0
    matric=w.T.dot(b)
    for i in range(0,data.shape[0]):
        for j in range(0,data.shape[1]):
            if data[i][j]!=0:
                totalError+=(data[i][j]-matric[i][j])**2
    totalError+=lamb*(w.dot(w.T)+b.dot(b.T))
    return totalError[0][0]

In [0]:
def rmse(data, Q, P):
    I = data != 0  # Indicator function which is zero for missing data
    ME = I * (data - np.dot(Q.T, P))  # Errors between real and predicted ratings
    MSE = ME**2  
    return np.sqrt(np.sum(MSE)/np.sum(I))  # sum of squared errors

In [0]:
def minbatch_gradient_descent_runner1(data,starting_w,starting_b,minbatch,learning_rate,lamb,max_iterations):
    b=starting_b
    w=starting_w
    lasterror = 0
    row,column=data.nonzero()
    zero=np.zeros(data.shape)
    partial_root_mean_square_error=[]
    
    random.seed(10)
    group_index=[[row[l],column[l]] for l in range(len(row))]
    v=0
    group=[]
    while v < row.shape[0]:
      group.append(group_index[v:v+minbatch])
      v+=minbatch
    group=np.array(group)
    ##group[k][l][m]=> k-th group l-th data m=0 =>row m=1 =>column
    
    for j in range(max_iterations):
      for i in range(0,group.shape[0]):
        zero=np.zeros(data.shape)
        w,b=minbatch_partial_step_gradient1(w,b,data,i,learning_rate,lamb)
      partial_root_mean_square_error.append(rmse(data, w, b))
      print("The {}-th iteration rmse = {}".format(j+1,rmse(data, w, b)))
      print("------------------------------")
      if np.abs(error(w,b,lamb,data)-lasterror)>0.1:
        lasterror=error(w,b,lamb,data)
      else:
        break
    return partial_root_mean_square_error

In [0]:
minbatch=10000
w=np.array([[2]*data.shape[0]])
b=np.array([[3]*data.shape[1]])
lamb=0.01
learning_rate=0.0004
max_iterations=300
row,column=data.nonzero()
minbatch_root_mean_square_error=minbatch_gradient_descent_runner1(data,w,b,minbatch,learning_rate,lamb,max_iterations)

The 1-th iteration rmse = 1.6329030236470643
------------------------------
The 2-th iteration rmse = 1.549023943312436
------------------------------
The 3-th iteration rmse = 1.6595376726635387
------------------------------
The 4-th iteration rmse = 1.890747261958489
------------------------------
The 5-th iteration rmse = 2.2082854003047205
------------------------------
The 6-th iteration rmse = 2.521898379497624
------------------------------
The 7-th iteration rmse = 2.693303242412501
------------------------------
The 8-th iteration rmse = 2.5244794464540354
------------------------------
The 9-th iteration rmse = 2.0716830665939248
------------------------------
The 10-th iteration rmse = 1.5497834415816243
------------------------------
The 11-th iteration rmse = 1.2329619109244119
------------------------------
The 12-th iteration rmse = 1.091902656529898
------------------------------
The 13-th iteration rmse = 1.0386485778023238
------------------------------
The 14-th ite

## **Gradient Descent**

In [0]:
def residual(data,w,b):
    I = data != 0  # Indicator function which is zero for missing data
    ME = I * (data - np.dot(w.T, b))  # Errors between real and predicted ratings  
    return  ME

In [0]:
def step_gradient1(w,b,data,learning_rate,lamb):
    w_gradient=(residual(data,w,b).dot(b.T).T-w*lamb)
    b_gradient=(residual(data,w,b).T.dot(w.T).T-b*lamb)
    new_w=w+(w_gradient*learning_rate)
    new_b=b+(b_gradient*learning_rate)
    return [new_w,new_b]

In [0]:
def error(w,b,lamb,data):
    totalError=0
    matric=w.T.dot(b)
    for i in range(0,data.shape[0]):
        for j in range(0,data.shape[1]):
            if data[i][j]!=0:
                totalError+=(data[i][j]-matric[i][j])**2
    totalError+=lamb*(w.dot(w.T)+b.dot(b.T))
    return totalError[0][0]

In [0]:
def rmse(data, Q, P):
    I = data != 0  # Indicator function which is zero for missing data
    ME = I * (data - np.dot(Q.T, P))  # Errors between real and predicted ratings
    MSE = ME**2  
    return np.sqrt(np.sum(MSE)/np.sum(I))  # sum of squared errors

In [0]:
def gradient_descent_runner1(data,starting_w,starting_b,learning_rate,lamb,max_iterations):
    b=starting_b
    w=starting_w
    lasterror = 0
    root_mean_square_error=[]
    for i in range(max_iterations):
        w,b=step_gradient1(w,b,data,learning_rate,lamb)
        root_mean_square_error.append(rmse(data, w, b))
        print("The {}-th iteration rmse = {}".format(i+1,rmse(data, w, b)))
        print("------------------------------")
        if np.abs(error(w,b,lamb,data)-lasterror)>0.1:
          lasterror=error(w,b,lamb,data)
        else:
          break
    return root_mean_square_error

In [0]:
w=np.array([[2]*data.shape[0]])
b=np.array([[3]*data.shape[1]])
learning_rate=0.0004
lamb=0.01
max_iterations=300
root_mean_square_error=gradient_descent_runner1(data,w,b,learning_rate,lamb,max_iterations)


The 1-th iteration rmse = 1.638008102859314
------------------------------
The 2-th iteration rmse = 1.5721357439436643
------------------------------
The 3-th iteration rmse = 1.7127699215283552
------------------------------
The 4-th iteration rmse = 1.9646243944425752
------------------------------
The 5-th iteration rmse = 2.299471049623312
------------------------------
The 6-th iteration rmse = 2.594620940215116
------------------------------
The 7-th iteration rmse = 2.7111985559629215
------------------------------
The 8-th iteration rmse = 2.4493117016333765
------------------------------
The 9-th iteration rmse = 1.9496050048977474
------------------------------
The 10-th iteration rmse = 1.4570512941563194
------------------------------
The 11-th iteration rmse = 1.1915658978945243
------------------------------
The 12-th iteration rmse = 1.078811060535889
------------------------------
The 13-th iteration rmse = 1.0346627648374405
------------------------------
The 14-th it

# **With Global Average Item Bias User Bias User-Item Interaction**



$ \hat\gamma_{ui} = \mu + b_i +b_u+ q^T _i  p_u  $

$min_{q^*,p^*,b^*}\sum_{(u,i)\in\kappa }{(\gamma_{ui}-\mu-b_i-b_u-q^T _i  p_u)^2} +\lambda (||q_i||^2+||p_u||^2+b^2_i+b^2_u)$

其中：

$b_i $: item biases $\quad b_u$ : user biases $\quad \mu$ : overall average

$\gamma$ : user-item rating  $\quad q_i$ :  item possesses those factors $\quad p_u$ :  interest the user has in items that are high on the corresponding factors

## **With Gradient Descent**

$e_{ui}=\gamma_{ui} - \mu-b_i-b_u-q^T _i  p_u $
*   $b_i \leftarrow b_i+\gamma \cdot (e_{ui}\cdot 1 - \lambda \cdot b_i)$
*   $b_u \leftarrow b_u+\gamma \cdot (e_{ui}\cdot 1 - \lambda \cdot b_u)$
*   $q_i \leftarrow q_i+\gamma \cdot (e_{ui}\cdot p_u - \lambda \cdot q_i)$
*   $p_u \leftarrow p_u+\gamma \cdot (e_{ui}\cdot q_i - \lambda \cdot p_u)$

其中 $\gamma$ : learning rate

In [0]:
def mu(data):
    I = data != 0  # Indicator function which is zero for missing data
    ME=I*data #calculate the rating
    return (np.sum(ME)/np.sum(I))  # mean

In [0]:
def residual_mean(data,w,b,mean_w,mean_b):
  I = data != 0
  mean_w_matric=mean_w.T.dot(np.array([[1]*data.shape[1]]))
  mean_b_matric=np.array([[1]*data.shape[0]]).T.dot(mean_b)
  mu_matric=mu(data)*np.dot(np.array([[1]*data.shape[0]]).T,np.array([[1]*data.shape[1]]))
  ME = I * (data -mu_matric-mean_w_matric-mean_b_matric- np.dot(w.T, b))
  return ME

In [0]:
def error_mean(data,w,b,mean_w,mean_b,lamb):
    totalError=0
    matric=w.T.dot(b)
    mean_w_matric=mean_w.T.dot(np.array([[1]*data.shape[1]]))
    mean_b_matric=np.array([[1]*data.shape[0]]).T.dot(mean_b)
    mu_data=mu(data)
    for i in range(0,data.shape[0]):
        for j in range(0,data.shape[1]):
            if data[i][j]!=0:
                totalError+=(data[i][j]-mu_data-mean_w_matric[i][j]-mean_b_matric[i][j]-matric[i][j])**2
    totalError+=lamb*(w.dot(w.T)+b.dot(b.T)+mean_w.dot(mean_w.T)+mean_b.dot(mean_b.T))
    return totalError[0][0]

In [0]:
def step_gradient_mean(w,b,mean_w,mean_b,data,learning_rate,lamb):
    w_gradient=(residual_mean(data,w,b,mean_w,mean_b).dot(b.T).reshape(1,w.shape[1])-w*lamb)
    b_gradient=(residual_mean(data,w,b,mean_w,mean_b).T.dot(w.T).reshape(1,b.shape[1])-b*lamb)
    mean_w_gradient=(residual_mean(data,w,b,mean_w,mean_b).dot(np.array([[1]*data.shape[1]]).T).reshape(1,mean_w.shape[1])-mean_w*lamb)
    mean_b_gradient=(residual_mean(data,w,b,mean_w,mean_b).T.dot(np.array([[1]*data.shape[0]]).T).reshape(1,mean_b.shape[1])-mean_b*lamb)
    new_w=w+(w_gradient*learning_rate)
    new_b=b+(b_gradient*learning_rate)
    new_mean_w=mean_w+(mean_w_gradient*learning_rate)
    new_mean_b=mean_b+(mean_b_gradient*learning_rate)
    return [new_w,new_b,new_mean_w,new_mean_b]

In [0]:
def rmse_mean(data, w, b,mean_w,mean_b):
    mean_w_matric=mean_w.T.dot(np.array([[1]*data.shape[1]]))
    mean_b_matric=np.array([[1]*data.shape[0]]).T.dot(mean_b)
    mu_matric=mu(data)*np.dot(np.array([[1]*data.shape[0]]).T,np.array([[1]*data.shape[1]]))
    I = data != 0  # Indicator function which is zero for missing data
    ME = I * (data-mu_matric -mean_w_matric-mean_b_matric- np.dot(w.T, b))  # Errors between real and predicted ratings
    MSE = ME**2  
    return np.sqrt(np.sum(MSE)/np.sum(I)) 

In [0]:
def gradient_descent_runner_mean(data,starting_w,starting_b,starting_mean_w,starting_mean_b,learning_rate,lamb,max_iterations):
    b=starting_b
    w=starting_w
    mean_w=starting_mean_w
    mean_b=starting_mean_b
    lasterror = 0
    rmse_list=[]
    for i in range(max_iterations):
        w,b,mean_w,mean_b=step_gradient_mean(w,b,mean_w,mean_b,data,learning_rate,lamb)
        rmse_list.append(rmse_mean(data, w, b,mean_w,mean_b))
        print("The {}-th iteration rmse = {}".format(i+1,rmse_mean(data, w, b,mean_w,mean_b)))
        print("------------------------------")
        if np.abs(error_mean(data,w,b,mean_w,mean_b,lamb)-lasterror)>0.1:
          lasterror=error_mean(data,w,b,mean_w,mean_b,lamb)
        else:
          break
    return rmse_list

In [0]:
learning_rate=0.0004
lamb=0.01
w=np.array([[-0.1]*data.shape[0]])
b=np.array([[0.2]*data.shape[1]])
mean_w=np.array([[-0.1]*data.shape[0]])
mean_b=np.array([[0.1]*data.shape[1]])
max_iterations=300
root_mean_square_error_mean=gradient_descent_runner_mean(data,w,b,mean_w,mean_b,learning_rate,lamb,max_iterations)

The 1-th iteration rmse = 1.0954603507443303
------------------------------
The 2-th iteration rmse = 1.0727784690733126
------------------------------
The 3-th iteration rmse = 1.0552609488382696
------------------------------
The 4-th iteration rmse = 1.0413367155133255
------------------------------
The 5-th iteration rmse = 1.0300016405054941
------------------------------
The 6-th iteration rmse = 1.020590955945247
------------------------------
The 7-th iteration rmse = 1.0126490297902806
------------------------------
The 8-th iteration rmse = 1.0058534345501153
------------------------------
The 9-th iteration rmse = 0.9999694962741976
------------------------------
The 10-th iteration rmse = 0.9948222401485397
------------------------------
The 11-th iteration rmse = 0.9902785034805677
------------------------------
The 12-th iteration rmse = 0.9862351627212789
------------------------------
The 13-th iteration rmse = 0.9826111524724035
------------------------------
The 14-th

## **With Min-batch**

In [0]:
def mu(data):
    I = data != 0  # Indicator function which is zero for missing data
    ME=I*data #calculate the rating
    return (np.sum(ME)/np.sum(I))  # mean

In [0]:
def minbatch_partial_residual_mean(data,i,minbatch,w,b,mean_w,mean_b):
  random.seed(10)
  group_index=[[row[l],column[l]] for l in range(len(row))]
  v=0
  group=[]
  while v < row.shape[0]:
    group.append(group_index[v:v+minbatch])
    v+=minbatch
  group=np.array(group)
  ##group[k][l][m]=> k-th group l-th data m=0 =>row m=1 =>column

  zero=np.zeros(data.shape)
  data_zero=np.zeros(data.shape)
  for k in range(group[i].shape[0]):
    zero[group[i][k][0]][group[i][k][1]]=1
    data_zero[group[i][k][0]][group[i][k][1]]=data[group[i][k][0]][group[i][k][1]]

  mean_w_matric=mean_w.T.dot(np.array([[1]*data.shape[1]]))
  mean_b_matric=np.array([[1]*data.shape[0]]).T.dot(mean_b)
  mu_matric=mu(data)*np.dot(np.array([[1]*data.shape[0]]).T , np.array([[1]*data.shape[1]]))
  minbatch_partial_residual = zero * (data_zero -mu_matric-mean_w_matric-mean_b_matric- np.dot(w.T, b))
  return minbatch_partial_residual

In [0]:
def minbatch_step_gradient_mean(w,b,mean_w,mean_b,data,i,minbatch,learning_rate,lamb):
    w_gradient=(minbatch_partial_residual_mean(data,i,minbatch,w,b,mean_w,mean_b).dot(b.T).T-w*lamb)
    b_gradient=(minbatch_partial_residual_mean(data,i,minbatch,w,b,mean_w,mean_b).T.dot(w.T).T-b*lamb)
    mean_w_gradient=(minbatch_partial_residual_mean(data,i,minbatch,w,b,mean_w,mean_b).dot(np.array([[1]*data.shape[1]]).T).T-mean_w*lamb)
    mean_b_gradient=(minbatch_partial_residual_mean(data,i,minbatch,w,b,mean_w,mean_b).T.dot(np.array([[1]*data.shape[0]]).T).T-mean_b*lamb)
    new_w=w+(w_gradient*learning_rate)
    new_b=b+(b_gradient*learning_rate)
    new_mean_w=mean_w+(mean_w_gradient*learning_rate)
    new_mean_b=mean_b+(mean_b_gradient*learning_rate)
    return [new_w,new_b,new_mean_w,new_mean_b]

In [0]:
def error_mean(data,w,b,mean_w,mean_b,lamb):
    totalError=0
    matric=w.T.dot(b)
    mean_w_matric=mean_w.T.dot(np.array([[1]*data.shape[1]]))
    mean_b_matric=np.array([[1]*data.shape[0]]).T.dot(mean_b)
    mu_data=mu(data)
    for i in range(0,data.shape[0]):
        for j in range(0,data.shape[1]):
            if data[i][j]!=0:
                totalError+=(data[i][j]-mu_data-mean_w_matric[i][j]-mean_b_matric[i][j]-matric[i][j])**2
    totalError+=lamb*(w.dot(w.T)+b.dot(b.T)+mean_w.dot(mean_w.T)+mean_b.dot(mean_b.T))
    return totalError[0][0]

In [0]:
def rmse_mean(data, w, b,mean_w,mean_b):
    mean_w_matric=mean_w.T.dot(np.array([[1]*data.shape[1]]))
    mean_b_matric=np.array([[1]*data.shape[0]]).T.dot(mean_b)
    mu_matric=mu(data)*np.dot(np.array([[1]*data.shape[0]]).T,np.array([[1]*data.shape[1]]))
    I = data != 0  # Indicator function which is zero for missing data
    ME = I * (data-mu_matric -mean_w_matric-mean_b_matric- np.dot(w.T, b))  # Errors between real and predicted ratings
    MSE = ME**2  
    return np.sqrt(np.sum(MSE)/np.sum(I)) 

In [0]:
def minbatch_gradient_descent_runner_mean(data,starting_w,starting_b,starting_mean_w,starting_mean_b,minbatch,learning_rate,lamb,max_iterations):
    b=starting_b
    w=starting_w
    mean_w=starting_mean_w
    mean_b=starting_mean_b
    lasterror = 0
    rmse_list=[]

    ##put data sample without replacement into group
    random.seed(10)
    group_index=[[row[l],column[l]] for l in range(len(row))]
    v=0
    group=[]
    while v < row.shape[0]:
      group.append(group_index[v:v+minbatch])
      v+=minbatch
    group=np.array(group)
    ##group[k][l][m]=> k-th group l-th data m=0 =>row m=1 =>column
    
    for j in range(0,max_iterations):    
      for i in range(0,group.shape[0]):
        zero=np.zeros(data.shape)
        w,b,mean_w,mean_b=minbatch_step_gradient_mean(w,b,mean_w,mean_b,data,i,minbatch,learning_rate,lamb)
      rmse_list.append(rmse_mean(data, w, b,mean_w,mean_b))
      print("The {}-th iteration rmse = {}".format(j+1,rmse_mean(data, w, b,mean_w,mean_b)))
      print("------------------------------")
      if np.abs(error_mean(data,w,b,mean_w,mean_b,lamb)-lasterror)>0.1:
          lasterror=error_mean(data,w,b,mean_w,mean_b,lamb)
      else:
        break
    return rmse_list

In [0]:
learning_rate=0.0004
lamb=0.01
w=np.array([[-0.1]*data.shape[0]])
b=np.array([[0.2]*data.shape[1]])
mean_w=np.array([[-0.1]*data.shape[0]])
mean_b=np.array([[0.1]*data.shape[1]])
max_iterations=300
minbatch=10000
row,column=data.nonzero()
minbatch_root_mean_square_error_mean=minbatch_gradient_descent_runner_mean(data,w,b,mean_w,mean_b,minbatch,learning_rate,lamb,max_iterations)

The 1-th iteration rmse = 1.096035349994559
------------------------------
The 2-th iteration rmse = 1.0736575539657933
------------------------------
The 3-th iteration rmse = 1.056279062270942
------------------------------
The 4-th iteration rmse = 1.0423965872394305
------------------------------
The 5-th iteration rmse = 1.0310481829717755
------------------------------
The 6-th iteration rmse = 1.02159476370301
------------------------------
The 7-th iteration rmse = 1.0135960221154101
------------------------------
The 8-th iteration rmse = 1.006738468750487
------------------------------
The 9-th iteration rmse = 1.000792482726094
------------------------------
The 10-th iteration rmse = 0.9955858081623129
------------------------------
The 11-th iteration rmse = 0.9909866178984699
------------------------------
The 12-th iteration rmse = 0.9868923184581162
------------------------------
The 13-th iteration rmse = 0.9832219177722757
------------------------------
The 14-th iter

# **Plot**

In [0]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(1,301), y=minbatch_root_mean_square_error,
                    mode='lines+markers',
                    name='Plain-Mini-batch'))
fig.add_trace(go.Scatter(x=np.arange(1,301), y=root_mean_square_error,
                    mode='lines+markers',
                    name='Plain-Gradient Descent'))
fig.add_trace(go.Scatter(x=np.arange(1,301), y=minbatch_root_mean_square_error_mean,
                    mode='lines+markers',
                    name='With Biases-Mini-batch'))
fig.add_trace(go.Scatter(x=np.arange(1,301), y=root_mean_square_error_mean,
                    mode='lines+markers',
                    name='With Biases-Gradient Descent'))
fig.update_layout(
    title="Movielens-100k Matrix Factorization",
    xaxis_title="Iterations",
    yaxis_title="RMSE",
)
fig.show()